In [ ]:
#import skimage.io

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm

import scipy.signal

In [ ]:
from util import plotstyle

In [ ]:
plotstyle.load('screen_dark')
#plotstyle.load('pretty_print')
#plotstyle.load('print')

In [ ]:
from util import bfp

# Calibrate using a Calibration Dataset
## Create Wrapper Object

In [ ]:
calibration_dataset = bfp.SliceDataset()

## Load Image Data

In [ ]:
calibration_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211111/calibration_test_01.tif") )

In [ ]:
calibration_dataset.img_data_avg.shape

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=100)

axs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( np.clip( calibration_dataset.img_data_avg, 0.00003, 1), 
                norm=LogNorm(vmax=1), 
                cmap=plotstyle.cmap('a'))
ax.set_xlabel('$x$ [px]')
ax.set_ylabel('$y$ [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Cumulative sCMOS Image")

plt.tight_layout()
plt.show()

## Generate Mask

In [ ]:
calibration_dataset.gen_mask()

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=100)

axs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( calibration_dataset.img_mask, cmap=plotstyle.cmap('bin'))
ax.set_xlabel('$x$ [px]')
ax.set_ylabel('$y$ [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Mask")

plt.tight_layout()
plt.show()

## Generate Regions of Interest from the Mask

In [ ]:
calibration_dataset.gen_rois()

In [ ]:
[ roi.to_string() for roi in calibration_dataset.rois ]

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int(np.sum(np.array([ r.width() for r in calibration_dataset.rois ])))+len(calibration_dataset.rois) )

for i in range(len(calibration_dataset.rois)):
    ax = fig.add_subplot(
        axs[ 0 , int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i] ])))+i:int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i+1] ])))+i ]
    )
    im = ax.imshow( 
        calibration_dataset.rois[i](calibration_dataset.img_data_avg), 
        cmap=plotstyle.cmap('a'), vmin=0, aspect='auto',
        extent=calibration_dataset.rois[i].to_extent()
    )
    if i==0:
        ax.set_ylabel('$y$ [px]')
    else:
        ax.set(yticklabels=[])
    if len(calibration_dataset.rois)//2 == i:
        ax.set_xlabel('$x$ [px]')
    ax.set_title("ROI {i}".format(i=i))

#plt.tight_layout()
plt.show()

## Compute the Calibration Constant

In [ ]:
calibration = calibration_dataset.get_calibration()

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int(np.sum(np.array([ r.width() for r in calibration_dataset.rois ])))+len(calibration_dataset.rois) )

for i in range(len(calibration_dataset.rois)):
    ax = fig.add_subplot(
        axs[ 0 , int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i] ])))+i:int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i+1] ])))+i ]
    )
    EXTENT = calibration_dataset.rois[i].to_extent()
    EXTENT[0] -= calibration_dataset.rois[0].get_xmean()
    EXTENT[1] -= calibration_dataset.rois[0].get_xmean()
    im = ax.imshow( 
        calibration_dataset.rois[i](calibration_dataset.img_data_avg), 
        cmap=plotstyle.cmap('a'), vmin=0, aspect='auto',
        extent=EXTENT
    )
    ax.fill_betweenx( 
        #np.arange(calibration_dataset.roi_mids.shape[1]),
        calibration_dataset.rois[i].Y(),
        #np.convolve( calibration_dataset.roi_mids[i]-calibration_dataset.roi_devs[i], np.array([1,1,1,1,1])/5, mode='same' )+calibration_dataset.rois[i].xmin, 
        #np.convolve( calibration_dataset.roi_mids[i]+calibration_dataset.roi_devs[i], np.array([1,1,1,1,1])/5, mode='same' )+calibration_dataset.rois[i].xmin, 
        calibration_dataset.roi_mids[i] - calibration_dataset.roi_devs[i] - calibration_dataset.rois[0].get_xmean(), 
        calibration_dataset.roi_mids[i] + calibration_dataset.roi_devs[i] - calibration_dataset.rois[0].get_xmean(), 
        color='r', alpha=2*plotstyle.err_alpha() )
    #ax.plot( 
    #    calibration_dataset.roi_mids[i] + calibration_dataset.rois[i].xmin, 
    #    np.arange(calibration_dataset.roi_mids.shape[1]),
    #    color='r', lw=1 )
    ax.plot( 
        [ np.mean(calibration_dataset.roi_mids[i])+calibration_dataset.rois[i].xmin-calibration_dataset.rois[0].get_xmean(), 
          np.mean(calibration_dataset.roi_mids[i])+calibration_dataset.rois[i].xmin-calibration_dataset.rois[0].get_xmean()], 
        [ -0.5, calibration_dataset.roi_mids.shape[1]-0.5 ],
        color=plotstyle.monochrome_fg(), lw=1, ls=':' )
    
    #limits = calibration_dataset.rois[i].to_extent()
    ax.set_xlim( xmin=EXTENT[0], xmax=EXTENT[1] )
    ax.set_ylim( ymin=EXTENT[2], ymax=EXTENT[3] )
    #del limits
    
    if i==0:
        ax.set_ylabel('$y$ [px]')
    else:
        ax.set(yticklabels=[])
    if len(calibration_dataset.rois)//2 == i:
        ax.set_xlabel('$\Delta x$ [px]')
    ax.set_title("ROI {i}".format(i=i))

#del limits
del EXTENT

#plt.tight_layout()
plt.show()

In [ ]:
for roi in calibration_dataset.rois:
    print( roi.to_string() )

In [ ]:
print( calibration.to_dict() )

In [ ]:
print( 
    "{a:.3f} nm/px ± {da:.4f} nm/px\n{b:.3f} px/nm ± {db:.4f} px/nm".format( 
        a=calibration.px_to_lda(), da=calibration.lda_error(),
        b=calibration.lda_to_px(), db=calibration.px_error() 
    ) 
)

In [ ]:
#print("Calibration Done. Press [ENTER] to continue.")

In [ ]:
#input()

# Extract a Spectral Correction from one Slice Image

In [ ]:
slice_dataset = bfp.SliceDataset()
#slice_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211112/scatterer01/lineprofile-12500µm.tif") )
slice_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211112/lamp-spectrum-01.tif") )


In [ ]:
slice_dataset.gen_mask()


In [ ]:
fig = plt.figure(figsize=(11,11), dpi=100)

axs = fig.add_gridspec(2, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( np.clip( slice_dataset.img_data_avg, 0.00003, 1), 
                norm=LogNorm(vmax=1), 
                cmap=plotstyle.cmap('a'))
ax.set_xlabel('$x$ [px]')
ax.set_ylabel('$y$ [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Cumulative sCMOS Image")

ax = fig.add_subplot(axs[ 1 , 0 ])

im = ax.imshow( slice_dataset.img_mask, cmap=plotstyle.cmap('bin'))
ax.set_xlabel('$x$ [px]')
ax.set_ylabel('$y$ [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Mask")

plt.tight_layout()
plt.show()

In [ ]:
slice_dataset.gen_rois()

In [ ]:
slice_dataset.map_rois()

In [ ]:
slice_dataset.correct_top_bottom_bg()

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int( slice_dataset.SLICE.width() + slice_dataset.SMEAR.width() ) + 10 )

EXTENT = slice_dataset.SLICE.to_extent()
EXTENT[0] -= slice_dataset.SLICE.xmid()
EXTENT[1] -= slice_dataset.SLICE.xmid()

ax = fig.add_subplot( axs[ 0 , :int( slice_dataset.SLICE.width() ) ] )
im = ax.imshow( 
    slice_dataset.SLICE(slice_dataset.img_data_avg), 
    #norm=LogNorm(vmax=1, vmin=0.0001), 
    vmin=0,
    cmap=plotstyle.cmap('a'), aspect='auto',
    extent=EXTENT
)
ax.set_ylabel('$y$ [px]')
ax.set_xlabel('$\Delta x$ [px]')
ax.set_title("SLICE")

ax.grid()



EXTENT = slice_dataset.SMEAR.to_extent()
EXTENT[0] -= slice_dataset.x0_px
EXTENT[1] -= slice_dataset.x0_px
EXTENT[0] *= calibration.px_to_lda()
EXTENT[1] *= calibration.px_to_lda()

ax = fig.add_subplot( axs[ 0 , int( slice_dataset.SLICE.width() )+10: ] )
im = ax.imshow( 
    slice_dataset.SMEAR(slice_dataset.img_data_avg), 
    #norm=LogNorm(vmax=1, vmin=0.0001), 
    vmin=0,
    cmap=plotstyle.cmap('a'), aspect='auto',
    extent=EXTENT
)
ax.set(yticklabels=[])
ax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')
ax.set_title("SMEAR")

ax.grid()
    
del EXTENT
    
#plt.tight_layout()
plt.show()

In [ ]:
#lineweights = np.sum( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=1 )
#lineweights /= np.sum(lineweights)

#mean_spec = np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 )
#mean_spec /= np.max(mean_spec)


In [ ]:
fig = plt.figure(figsize=(12,9), dpi=100)

axs = fig.add_gridspec( 1, 1 )

ax = fig.add_subplot( axs[ 0 , : ] )

BINNING = 4
for i in np.arange( 0, slice_dataset.SMEAR(slice_dataset.img_data_avg).shape[0], BINNING ):
    ax.plot(
        calibration.px_to_lda( slice_dataset.SMEAR.X()-slice_dataset.x0_px ),
        np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg)[i:i+BINNING], axis=0)/np.max(np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg)[i:i+BINNING], axis=0)),
        color=plotstyle.c(0), #monochrome_fg(),
        lw=1, ls='-', alpha=np.max(np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg)[i:i+BINNING], axis=0))
    )
del BINNING

ax.plot(
    calibration.px_to_lda( slice_dataset.SMEAR.X()-slice_dataset.x0_px ),
    np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 )/np.max(np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 )),
    color=plotstyle.monochrome_fg()    
)

#ax.set(yticklabels=[])
ax.set_ylabel('Rel. Spectral Efficiency [a.u.]')
ax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')
ax.set_title("Spectral Correction")

ax.grid()
    

    
plt.tight_layout()
plt.show()

## Generate an Interpolation Function for the Spectral Correction

In [ ]:
from scipy import interpolate

In [ ]:
LDA = calibration.px_to_lda( slice_dataset.SMEAR.X()-slice_dataset.x0_px )
VAL = np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 )/np.max(np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 ))

In [ ]:
correction = scipy.interpolate.interp1d( LDA, VAL, kind=2 )

In [ ]:
del LDA
del VAL

# Conversion of Offset on the Slit to Pixel Offset

In [ ]:
M2 = 2.0 # magnification from slit to sensor
px_size = 6.5 # micrometres
um_to_px_offset = M2/px_size
print( "{:.3f} px/µm".format(um_to_px_offset) )
print( "Offset @ 50 µm: {:.3f} px".format(50*um_to_px_offset) )

# Building a full 3D BFP Dataset

In [ ]:
shifts_um = np.arange( 11850, 13150+50, 50 )
files = [ "../../labdata/TIFF/211112/scatterer01/lineprofile-{:}µm.tif".format(shift) for shift in shifts_um ]
files

In [ ]:
layers = shifts_um.shape[0]
im_height = 1024
shifts_px = np.round( um_to_px_offset*( shifts_um - np.min(shifts_um) ) ).astype(int)
#shifts_px = np.round( um_to_px_offset*( -shifts_um + np.min(shifts_um) ) ).astype(int)
#shifts_px = np.round( -um_to_px_offset*( shifts_um - np.max(shifts_um) ) ).astype(int)
#shifts_px *= -1
#shifts_px -= np.min(shifts_px)
# required width of the image stack, with some margin
im_width = np.max(shifts_px)-np.min(shifts_px)+5*np.max(np.diff(shifts_px))
shifts_px += np.max(np.diff(shifts_px))
#shifts_px -= np.max(np.diff(shifts_px))

images = np.zeros( (layers, im_height, im_width) )


In [ ]:
shifts_px = np.flipud(shifts_px) # TODO: do this properly, somehow through the offset factor

In [ ]:
#images.shape
shifts_px
#im_width

In [ ]:
slices = [ bfp.SliceDataset().load_image_data( bfp.TIFF_Stack().load_file( FILE ) ) for FILE in files ]


In [ ]:
for sl in slices:
    sl.gen_mask()
    sl.gen_rois()
    sl.map_rois()
    sl.correct_top_bottom_bg()



In [ ]:
fig = plt.figure(figsize=(12,6*len(slices)), dpi=100)

axs = fig.add_gridspec( len(slices), 10 )

i=0
for sl in slices:
    EXTENT = sl.SLICE.to_extent()
    EXTENT[0] -= sl.SLICE.xmid()
    EXTENT[1] -= sl.SLICE.xmid()

    ax = fig.add_subplot( axs[ i , :1 ] )
    im = ax.imshow( 
        sl.SLICE(sl.img_data_avg), 
        #norm=LogNorm(vmax=1, vmin=0.0001), 
        vmin=0,
        cmap=plotstyle.cmap('a'), aspect='auto',
        extent=EXTENT
    )
    ax.set_ylabel('$y$ [px]')
    ax.set_xlabel('$\Delta x$ [px]')
    ax.set_title("SLICE")

    ax.grid()



    EXTENT = sl.SMEAR.to_extent()
    EXTENT[0] -= sl.x0_px
    EXTENT[1] -= sl.x0_px
    EXTENT[0] *= calibration.px_to_lda()
    EXTENT[1] *= calibration.px_to_lda()

    ax = fig.add_subplot( axs[ i , 1: ] )
    im = ax.imshow( 
        sl.SMEAR(sl.img_data_avg), 
        #norm=LogNorm(vmax=1, vmin=0.0001), 
        vmin=0,
        cmap=plotstyle.cmap('a'), aspect='auto',
        extent=EXTENT
    )
    ax.set(yticklabels=[])
    ax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')
    ax.set_title("SMEAR")

    ax.grid()

    del EXTENT
    i += 1
del i
    
plt.tight_layout()
plt.show()

In [ ]:
for i in range(layers):
    print(slices[i].img_data_avg[:,slices[i].SLICE.xslice()].shape, " -> ", images[i,:,shifts_px[i]:shifts_px[i]+slices[i].SLICE.width()].shape )

In [ ]:
for i in range(layers):
    images[i,:,shifts_px[i]:shifts_px[i]+slices[i].SLICE.width()] = np.copy( slices[i].img_data_avg[:,slices[i].SLICE.xslice()] ) # normalization factor, have to keep track of inside SliceDataset

In [ ]:
fig = plt.figure(figsize=(12,12), dpi=100)

axs = fig.add_gridspec( 1, 10 )

ax = fig.add_subplot( axs[ : , : ] )
im = ax.imshow( 
    np.sum(images, axis=0), 
    #norm=LogNorm(vmax=1, vmin=0.0001), 
    vmin=0,
    cmap=plotstyle.cmap('a'), #aspect='auto',
    #extent=EXTENT
)
ax.set_ylabel('$y$ [px]')
ax.set_xlabel('$\Delta x$ [px]')
ax.set_title("Re-assembled Image")

ax.grid()
    
plt.tight_layout()
plt.show()